In [7]:
import pandas as pd
import simpy
import random
import folium
from geopy.distance import geodesic
from shapely.geometry import Point, LineString
import csv

# Define routes (major highways in Vidarbha)
routes = {
    "NH6": [
        (20.9952, 79.0013),  # Nagpur
        (20.7325, 78.5448),  # Wardha
        (20.5237, 77.8036),  # Amravati
        (20.7359, 76.5638)   # Akola
    ],
    "NH7": [
        (21.1458, 79.0882),  # Nagpur
        (21.4456, 79.3671),  # Bhandara
        (21.7649, 79.6498),  # Gondia
        (21.9042, 80.2458)   # Balaghat (MP border)
    ],
    "NH53": [
        (20.7025, 78.6022),  # Wardha
        (20.2787, 78.2757),  # Yavatmal
        (19.9512, 77.7859)   # Nanded
    ]
}

# Define toll locations
tolls = [
    ("Nagpur Toll Plaza", 20.9847, 79.0246),
    ("Wardha Toll Plaza", 20.7456, 78.5890),
    ("Amravati Toll Plaza", 20.5543, 77.8235),
    ("Akola Toll Plaza", 20.7248, 76.5831),
    ("Bhandara Toll Plaza", 21.4234, 79.3512),
    ("Gondia Toll Plaza", 21.7532, 79.6345),
    ("Yavatmal Toll Plaza", 20.3012, 78.2901)
]

# Create routes DataFrame
routes_data = []
for route, coordinates in routes.items():
    for seq, (lat, lon) in enumerate(coordinates, 1):
        routes_data.append([route, seq, lat, lon])
routes_df = pd.DataFrame(routes_data, columns=["Route", "Sequence", "Latitude", "Longitude"])

# Create tolls DataFrame
tolls_df = pd.DataFrame(tolls, columns=["Name", "Latitude", "Longitude"])

class Vehicle:
    def __init__(self, env, vehicle_id, route, color, avoid_tolls=False, initial_balance=1000, cost_per_km=2.2):
        self.env = env
        self.vehicle_id = vehicle_id
        self.route = route
        self.color = color
        self.avoid_tolls = avoid_tolls
        self.current_position = 0
        self.coordinates = routes_df[routes_df['Route'] == route][['Latitude', 'Longitude']].values
        self.route_points = []
        self.balance = initial_balance
        self.cost_per_km = cost_per_km
        self.total_distance = 0
        self.total_toll_paid = 0
        self.start_point = self.coordinates[0]
        self.end_point = self.coordinates[-1]
        
        # Adjust route to avoid tolls if specified
        if self.avoid_tolls:
            self.coordinates = self.adjust_route_to_avoid_tolls(self.coordinates)
        
        self.route_points.append(self.coordinates[self.current_position])
        self.action = env.process(self.run())

    def adjust_route_to_avoid_tolls(self, coordinates):
        # Logic to remove coordinates that are close to tolls
        adjusted_route = []
        for i, (lat, lon) in enumerate(coordinates):
            if i == 0 or i == len(coordinates) - 1:
                # Always include start and end points
                adjusted_route.append((lat, lon))
                continue
            near_toll = False
            for _, toll_lat, toll_lon in tolls:
                if geodesic((lat, lon), (toll_lat, toll_lon)).km < 5:  # Assuming 5 km radius to consider near toll
                    near_toll = True
                    break
            if not near_toll:
                adjusted_route.append((lat, lon))
        return adjusted_route

    def run(self):
        while self.current_position < len(self.coordinates) - 1:
            start = self.coordinates[self.current_position]
            end = self.coordinates[self.current_position + 1]
            distance = geodesic(start, end).km
            self.total_distance += distance
            travel_time = distance * 10  # Assume travel time is 10 time units per km for simplicity
            toll_cost = distance * self.cost_per_km
            self.balance -= toll_cost
            self.total_toll_paid += toll_cost
            # Comment out or remove the following line to stop printing travel details
            # print(f'{self.vehicle_id} at {self.coordinates[self.current_position + 1]}: Travelled {distance} km, Toll cost {toll_cost} rupees, Remaining balance {self.balance} rupees')
            yield self.env.timeout(travel_time)  # Simulate travel time proportional to distance
            self.current_position += 1
            self.route_points.append(self.coordinates[self.current_position])

def setup(env, num_vehicles, vehicles):
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen']
    for i in range(num_vehicles):
        route = routes_df['Route'].sample().values[0]
        color = random.choice(colors)
        avoid_tolls = False
        if i == 0:  # First vehicle avoids tolls on NH6
            route = 'NH6'
            avoid_tolls = True
        vehicle = Vehicle(env, f'Vehicle {i+1}', route, color, avoid_tolls)
        vehicles.append(vehicle)
        yield env.timeout(1)  # Small delay between setting up each vehicle

# Initialize simulation environment and vehicles list
env = simpy.Environment()
vehicles = []

# Setup the simulation with vehicles
env.process(setup(env, 5, vehicles))  # Simulate 5 vehicles
env.run(until=100)  # Run the simulation for 100 time units

# Create a folium map centered around Vidarbha
map_vidarbha = folium.Map(location=[20.5937, 78.9629], zoom_start=8)

# Add routes to the map
for route, route_df in routes_df.groupby('Route'):
    line = [(row['Latitude'], row['Longitude']) for idx, row in route_df.iterrows()]
    folium.PolyLine(line, color='gray', weight=2.5, opacity=1).add_to(map_vidarbha)

# Add tolls to the map
for idx, row in tolls_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(map_vidarbha)

# Add vehicle routes with different colors
for vehicle in vehicles:
    # Add start point
    folium.Marker(
        location=vehicle.coordinates[0],
        popup=f'Start {vehicle.vehicle_id}',
        icon=folium.Icon(color='blue', icon='play')
    ).add_to(map_vidarbha)
    # Add end point
    folium.Marker(
        location=vehicle.coordinates[-1],
        popup=f'End {vehicle.vehicle_id}',
        icon=folium.Icon(color='black', icon='stop')
    ).add_to(map_vidarbha)
    # Add route line
    folium.PolyLine(vehicle.route_points, color=vehicle.color, weight=2.5, opacity=1).add_to(map_vidarbha)

# Save the map to an HTML file
map_vidarbha.save('vidarbha_map.html')

# Display the map
map_vidarbha

# Create a detailed report
report_data = []
for vehicle in vehicles:
    report_data.append({
        'Vehicle ID': vehicle.vehicle_id,
        'Route': vehicle.route,
        'Start Point': vehicle.start_point,
        'End Point': vehicle.end_point,
        'Total Distance Travelled (km)': round(vehicle.total_distance, 2),
        'Total Toll Paid (INR)': round(vehicle.total_toll_paid, 2),
        'Remaining Balance (INR)': round(vehicle.balance, 2)
    })

report_df = pd.DataFrame(report_data)

# Save the report to a CSV file
report_df.to_csv('vehicle_report.csv', index=False)

print("Simulation completed. Detailed report saved to 'vehicle_report.csv'.")


Simulation completed. Detailed report saved to 'vehicle_report.csv'.


In [8]:
map_vidarbha

In [9]:
report_df 

,Vehicle ID,Route,Start Point,End Point,Total Distance Travelled (km),Total Toll Paid (INR),Remaining Balance (INR)
0,Vehicle 1,NH6,"[20.9952, 79.0013]","[20.7359, 76.5638]",255.27,561.60,438.40
1,Vehicle 2,NH53,"[20.7025, 78.6022]","[19.9512, 77.7859]",57.98,127.55,872.45
2,Vehicle 3,NH6,"[20.9952, 79.0013]","[20.7359, 76.5638]",55.70,122.54,877.46
3,Vehicle 4,NH6,"[20.9952, 79.0013]","[20.7359, 76.5638]",55.70,122.54,877.46
4,Vehicle 5,NH7,"[21.1458, 79.0882]","[21.9042, 80.2458]",44.04,96.88,903.12
